<a href="https://colab.research.google.com/github/juneilsam/mariah_carey_christmas/blob/main/data_scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlopen
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
import time
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys



---

---





# 1. i-tunes 음원순위 데이터 날짜 수집

### 1-1. i-tunes 음원순위 데이터 날짜 수집

In [ ]:
# 리스트 가져오기

list_url = 'https://kworb.net/ww/archive/'
html = urlopen(list_url).read()
soup = BeautifulSoup(html, "lxml")
addresses = soup.find_all('a')

dates = []

for address in addresses:
    if len(address.text) >= 8:
        each = address.text[:8]
        dates.append(each)
    else:
        continue

### 1-2. 세계, 미국, 영국 순위 수집 (i- tunes)

In [ ]:
title = 'All I Want for Christmas Is You'
artist = 'Mariah Carey'
full = artist + ' - ' + title
daily = []

# 매일 순위 추가하기
def addRank(date):
    daily_url = f'{list_url}{date}.html'
    temp = []
    temp.append(str(date))
    
    # url open
    with urlopen(daily_url) as u:
            daily_soup = BeautifulSoup(u, "lxml")
            daily_table = daily_soup.find('table', {'class' : 'sortable'})
            
            try:
                daily_tr = daily_table.select('tr')
                artist_title = []
                rank = []
                rank2 = []
                
                # 아티스트 목록, US 순위, UK 순위
                for idx, tr in enumerate(daily_tr):                   
                    if idx >0:
                        artist_title.append(tr.select_one('td:nth-of-type(3)').text)
                        if len(tr.select_one('td:nth-of-type(9)').text) > 3:
                            rank.append(tr.select_one('td[style="border-left:1px solid black;"]').text)
                            rank2.append((tr.select_one('td[style="border-left:1px solid black;"]')).next_sibling.text)
                        else:
                            rank.append(tr.select_one('td:nth-of-type(9)').text)
                            rank2.append(tr.select_one('td:nth-of-type(9)').next_sibling.text)


                #데이터 추가
                if full in artist_title:
                    temp.append(str(artist_title.index(full) + 1))
                    temp.append(str(rank[artist_title.index(full)]))
                    temp.append(str(rank2[artist_title.index(full)]))

                else:
                    temp.extend(['400', '400', '400'])
                            
            except:
                temp.extend(['err', 'err', 'err'])
    
    daily.append(temp)
    
for date in tqdm(dates):
    addRank(date)

print(daily)

100%|██████████| 3608/3608 [1:29:06<00:00,  1.48s/it]

[['20110219', '400', '400', '400'], ['20110220', '400', '400', '400'], ['20110221', '400', '400', '400'], ['20110222', '400', '400', '400'], ['20110223', '400', '400', '400'], ['20110224', '400', '400', '400'], ['20110225', '400', '400', '400'], ['20110226', '400', '400', '400'], ['20110227', '400', '400', '400'], ['20110228', '400', '400', '400'], ['20110301', '400', '400', '400'], ['20110302', '400', '400', '400'], ['20110303', '400', '400', '400'], ['20110304', '400', '400', '400'], ['20110305', '400', '400', '400'], ['20110306', '400', '400', '400'], ['20110307', '400', '400', '400'], ['20110308', '400', '400', '400'], ['20110309', '400', '400', '400'], ['20110310', '400', '400', '400'], ['20110311', '400', '400', '400'], ['20110312', '400', '400', '400'], ['20110313', '400', '400', '400'], ['20110314', '400', '400', '400'], ['20110315', '400', '400', '400'], ['20110316', '400', '400', '400'], ['20110317', '400', '400', '400'], ['20110318', '400', '400', '400'], ['20110319', '400',

# 2. 미국, 영국 기온 정보 수집 

### 2-1. 일별 기온 데이터 수집을 위한 url 리스트

In [ ]:
dateT = []

# 기온 날짜 리스트
for year in range(2011, 2022):
    if year == 2011:
        for month in range(2, 13):
            date = str(year) + '-' + str(month)
            dateT.append(date)
            
    elif year == 2021:
        dateT.append('2021' + '-' + '1')
        
    else:
        for month in range(1, 13):
            date = str(year) + '-' + str(month)
            dateT.append(date)

### 2-2. 화씨 데이터를 섭씨 데이터로 변환하는 함수

In [ ]:
# 화씨-섭씨 변환 함수
def trans(f):
    c = (float(f) - 32) * 5 / 9
    return str(round(c, 2))

### 2-3. 미국(뉴욕), 영국(런던) 기온 데이터 수집

In [ ]:
dailyUS = []
dailyUK = []
us = 'us/ny/new-york-city/KLGA'
uk = 'gb/london/EGLC'

# 날씨 페이지 접속
def render_page(url):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', chrome_options=options)
    driver.get(url)
    time.sleep(5)
    r = driver.page_source
    driver.quit()
    return r


# 날씨 데이터 저장
def addClimate(country):
    
    for d in ['2011-02']:

        url = f'https://www.wunderground.com/history/monthly/{country}/date/{d}'

        # 3초 기다리는 함수 및 불러내기   
        r = render_page(url)

        soup = BeautifulSoup(r, "lxml")
        container = soup.select_one('lib-city-history-observation')
        check = container.select_one('tbody')
        table = check.select('table')[1]
        td = (i.select_one('td:nth-of-type(2)') for i in table)
        print(td.text)
        temp = []

        # 데이터 저장
        try:
            for i, data in enumerate(td):

                if d == '2011' + '-' + '2':
                    if i > 19:
                        temp.append(trans(data.text))

                elif i > 0:
                    temp.append(trans(data.text))


        except:
            temp.append('err')

        # 미국 날씨
        if country == us:
            dailyUS.append(temp)

        # 영국 날씨    
        else:
            dailyUK.append(temp)
     
        print(country + d + 'is done!')

for country in [us, uk]:
    addClimate(country)
    
print(dailyUS)
print(dailyUK)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


us/ny/new-york-city/KLGA2011-2is done!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


us/ny/new-york-city/KLGA2011-3is done!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


...

gb/london/EGLC2020-11is done!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


gb/london/EGLC2020-12is done!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


gb/london/EGLC2021-1is done!
[['-0.78', '-1.33', '-3.72', '-0.56', '2.28', '6.39', '2.11', '5.22', '8.11', 'err'], ['3.39', '4.33', '-3.22', '1.61', '9.17', '10.5', '5.17', '3.11', '3.0', '7.56', '9.67', '8.22', '7.94', '4.61', '3.67', '6.94', '11.39', '16.28', '7.94', '4.44', '5.11', '7.06', '1.61', '1.83', '1.44', '0.17', '1.33', '2.11', '4.11', '6.28', '4.22', 'err'], ['4.11', '8.17', '9.72', '10.56', '9.56', '7.22', '6.61', '6.22', '6.17', '9.33', '14.72', '12.33', '7.78', '13.33', '8.5', '8.5', '12.89', '11.39', '9.5', '9.44', '11.11', '6.94', '10.22', '18.22', '15.0', '15.72', '16.61', '16.78', '16.56', '13.67', 'err'], ['12.83', '12.67', '14.83', '11.28', '13.11', '16.06', '16.33', '16.67', '16.22', '15.83', '15.61', '15.17', '14.72', '15.22', '16.5', '12.94', '12.56', '13.78', '16.39', '15.83', '16.5', '13.28', '15.67', '22.5', '22.56', '20.94', '21.39', '22.44', '23.28', '25.17', '24.72', 'err'], ['26.22', '22.06', '19.0', '18.83', '16.22', '22.11', '26.78', '29.83', '27.33', 

### 2-4. 월별로 분리된 기온 데이터를 음원데이터와 합치기 위해 일별로 분리

In [ ]:
# 기온 정렬 함수
def openT(T):
    temp = []
    [temp.append(str(j)) for i in T for j in i]                      
    T, temp = temp, T    
    return T

### 2-5. 음원데이터와 기온 데이터 합치기

In [ ]:
b = daily[:]

print(openT(dailyUS))

# 정보 합치기, 순위 날짜 외 기온 제외
for n in range(len(daily)):
    b[n].insert(3, openT(dailyUS)[n])
    b[n].append(openT(dailyUK)[n])

print(b)

['2.78', '-0.78', '-1.33', '-3.72', '-0.56', '2.28', '6.39', '2.11', '5.22', '8.11', 'err', '3.39', '4.33', '-3.22', '1.61', '9.17', '10.5', '5.17', '3.11', '3.0', '7.56', '9.67', '8.22', '7.94', '4.61', '3.67', '6.94', '11.39', '16.28', '7.94', '4.44', '5.11', '7.06', '1.61', '1.83', '1.44', '0.17', '1.33', '2.11', '4.11', '6.28', '4.22', 'err', '4.11', '8.17', '9.72', '10.56', '9.56', '7.22', '6.61', '6.22', '6.17', '9.33', '14.72', '12.33', '7.78', '13.33', '8.5', '8.5', '12.89', '11.39', '9.5', '9.44', '11.11', '6.94', '10.22', '18.22', '15.0', '15.72', '16.61', '16.78', '16.56', '13.67', 'err', '12.83', '12.67', '14.83', '11.28', '13.11', '16.06', '16.33', '16.67', '16.22', '15.83', '15.61', '15.17', '14.72', '15.22', '16.5', '12.94', '12.56', '13.78', '16.39', '15.83', '16.5', '13.28', '15.67', '22.5', '22.56', '20.94', '21.39', '22.44', '23.28', '25.17', '24.72', 'err', '26.22', '22.06', '19.0', '18.83', '16.22', '22.11', '26.78', '29.83', '27.33', '25.22', '17.11', '17.33', '18

# 3. CSV 파일로 저장

In [ ]:
with open('mariah_xmas.csv','w',encoding='utf-8',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['date','WDRank','USRank', 'USTemp', 'UKRank', 'UKTemp'])
    writer.writerows(b)